In [1]:
import sys
import importlib
import os
# make the module available from the src directory
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from src.dataLoaders.PatientDataLoader import PatientDataProcessor
importlib.reload(sys.modules['src.dataLoaders.PatientDataLoader'])

<module 'src.dataLoaders.PatientDataLoader' from 'c:\\Datalogi\\Bachelor\\XAI-in-Medical-Image-Analysis\\src\\dataLoaders\\PatientDataLoader.py'>

In [3]:
meta_folder = 'OAIData21/'
data_path = '../datasets/meta_data/' + meta_folder

processor = PatientDataProcessor(base_path=data_path)
meta_data = processor.load_all_clinical_data(labels=["WOMKP"])

In [13]:
from src.dataLoaders.utils.BalancedDataLoader import BalancedBatchSampler
importlib.reload(sys.modules['src.dataLoaders.utils.BalancedDataLoader'])

from src.dataLoaders.NiftiDataLoader2 import NiftiDataLoader
importlib.reload(sys.modules['src.dataLoaders.NiftiDataLoader2'])

from src.transformers.SliceTransformer import SliceAggregateTransform
importlib.reload(sys.modules['src.transformers.SliceTransformer'])
custom_transforms = [SliceAggregateTransform(keys=["image"], slices=9)]

In [14]:
dataset = 'niftiShort'
data_path = '../datasets/nifti/'
data_dir = f'{data_path}{dataset}'

data_dir = 'C:/Users/askel/Downloads/NIFTY/NIFTY/'
# max = 8876
dim = 384
data_loader = NiftiDataLoader(data_dir=data_dir, 
                              meta_data_loader=processor,
                              batch_size=32,
                              spatial_resize=(dim, dim, dim),
                              cache_rate=0.5, 
                              replace_rate=1,
                              custom_transforms=custom_transforms,
                              custom_sampler=False
                              )
# data_loader.load_data(visit_no=visit, subset_size=2058, cache="standard")
# data_loader.load_data(subset_size=84, cache="standard")
#data_loader.load_data(subset_size=4400, cache="standard")
# data_loader.load_data(subset_size=8875, cache="standard")
# data_loader.load_data(cache="persistent")
data_loader.load_data(cache="standard", subset_size=1000)

File c:\Datalogi\Bachelor\XAI-in-Medical-Image-Analysis\src\dataLoaders\saved_data_lists\data_list.pkl does not exist.
Data list loaded: False
Using custom sampler: False
File c:\Datalogi\Bachelor\XAI-in-Medical-Image-Analysis\src\dataLoaders\saved_data_lists\data_list.pkl does not exist.
Visits: dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])
Total images detected: 8871
Subset size: 50


Loading dataset: 100%|██████████| 5/5 [00:05<00:00,  1.02s/it]

batch_size: 32


In [20]:
from src.models.medical_models.combined_medical import MedicalCombinedResNetModel
importlib.reload(sys.modules['src.models.medical_models.combined_medical'])

<module 'src.models.medical_models.combined_medical' from 'c:\\Datalogi\\Bachelor\\XAI-in-Medical-Image-Analysis\\src\\models\\medical_models\\combined_medical.py'>

In [21]:
combined_model = MedicalCombinedResNetModel(
        num_epochs=150,
        learning_rate=0.001,
        weight_decay=1e-4,
        data_loader=data_loader,
        dropout_rate=0.1,
        depth=18,
    )

combined_model.train()

Data loader train loader:  2 50
Image spatial dimensions:  torch.Size([32, 9, 384, 384])
Number of input channels:  9
gpu:  cuda:0
Is cuda available:  True cuda
Number of training images: 50
Epoch 1/150, Train Loss: 7.651394486427307, Val Loss: 0.6383402943611145, R^2 Score: 0.0025933384895324707
Model saved at c:\Datalogi\Bachelor\XAI-in-Medical-Image-Analysis\src\models\medical_models\saved_models\MedicalCombinedResNetModel_18_50_height_384_epoch_1_val_0.64_r2_0.64.pth
Epoch 2/150, Train Loss: 8.881916999816895, Val Loss: 30.40073585510254, R^2 Score: -46.50114822387695
Epoch 3/150, Train Loss: 4.706733465194702, Val Loss: 184.89500427246094, R^2 Score: -287.8984375
Epoch 4/150, Train Loss: 2.119087338447571, Val Loss: 156.6761016845703, R^2 Score: -243.806396484375
Epoch 5/150, Train Loss: 0.8830665349960327, Val Loss: 118.65670013427734, R^2 Score: -184.40109252929688
Epoch 6/150, Train Loss: 1.7883006036281586, Val Loss: 82.49055480957031, R^2 Score: -127.8914794921875


KeyboardInterrupt: 